<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/p123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import torch
import torchvision
import torchsummary
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d alenic/rsna-2023-atd-reduced-256-5mm
!unzip \*.zip && rm *.zip
clear_output()

In [9]:
# class AbdominalData(torch.utils.data.Dataset):
#     def __init__(self, df, current_fold, num_fold=5, train_folder=TRAIN_PATH, limit=-1):

#         super().__init__()
#         self.img_paths = []
#         for patient in tqdm(os.listdir(train_folder)[:limit]):
#             for scan in os.listdir(os.path.join(train_folder, patient)):
#                 scans = []
#                 for dicom_img in os.listdir(os.path.join(train_folder, patient, scan)):
#                     scans.append(os.path.join(train_folder, patient, scan, dicom_img))
#                 self.img_paths.append(scans)

#         self.df = df
#         self.current_fold = current_fold
#         self.kf = model_selection.KFold(n_splits=num_fold)
#         self.transform = torchvision.transforms.Compose([
#                             torchvision.transforms.Resize((256, 256), antialias=True),
#                             torchvision.transforms.RandomHorizontalFlip(),
#                             torchvision.transforms.ColorJitter(brightness=0.2),
#                             torchvision.transforms.ColorJitter(contrast=0.2),
#                             torchvision.transforms.RandomAffine(degrees=0, shear=10),
#                             torchvision.transforms.RandomAffine(degrees=0, scale=(0.8, 1.2)),
#                             torchvision.transforms.RandomErasing(p=0.2, scale=(0.02, 0.2)),
#         ])

#     def __len__(self):
#         return len(self.img_paths)

#     def __getitem__(self, idx, spacing = 2):

#         elements = self.img_paths[idx]

#         if len(elements) < spacing * 4:
#             raise ValueError("Wrong number of elements.")

#         # We want to select elements in the middle part of the abdomen
#         lower_bound = int(len(elements) * 0.4)
#         upper_bound = int(len(elements) * 0.6)

#         spacing = (upper_bound - lower_bound) // 3
#         selected_indices = [lower_bound, lower_bound + spacing, lower_bound + (2*spacing), upper_bound]
#         dicom_images = [elements[index] for index in selected_indices]

#         patient_id = dicom_images[0].split('/')[-3]
#         images = []
#         for d in dicom_images:
#             img = cv2.imread(d)
#             image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255
#             images.append(image)

#         images = np.stack(images)
#         images = torch.tensor(images, dtype = torch.float).unsqueeze(dim = 1)
#         image = self.transform(images).squeeze(dim = 1)
#         label = self.df[self.df.patient_id == int(patient_id)].values[0][1:]
#         return image, label

#     def get_splits(self):
#         fold_data = list(self.kf.split(self.img_paths))
#         train_indices, val_indices = fold_data[self.current_fold]
#         train_data = self._get_subset(train_indices)
#         val_data = self._get_subset(val_indices)
#         return train_data, val_data

#     def _get_subset(self, indices):
#         return torch.utils.data.Subset(self, indices)


# class AbdominalData(torch.utils.data.Dataset):
#     def __init__(self, df=train_data, train_folder=TRAIN_PATH, limit=-1):

#         super().__init__()
#         self.img_paths = []
#         for patient in tqdm(os.listdir(train_folder)):
#             for scan in os.listdir(os.path.join(train_folder, patient)):
#                 scans = []
#                 for dicom_img in os.listdir(os.path.join(train_folder, patient, scan)):
#                     scans.append(os.path.join(train_folder, patient, scan, dicom_img))
#                 self.img_paths.append(scans)

#         self.transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256, 256), antialias=True)])

#         print(self.img_paths)
#         for folder in self.img_paths:
#           images = []
#           patient_id = folder[0].split('/')[-3]
#           for elemnts in folder[:limit]:
#             img = cv2.imread(elemnts)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255
#             images.append(img)

#           images = np.stack(images)
#           images = torch.tensor(images, dtype = torch.float).unsqueeze(dim = 1)
#           image = self.transform(images).squeeze(dim = 1)
#           label = df[df.patient_id == int(patient_id)].values[0][1:]
#         return image, label

In [1]:
TRAIN_PATH = '/content/reduced_256_tickness_5'
train_data = pd.read_csv('./train.csv')
limit = 1
img_paths = []
img_list = []
label_list = []
for patient in tqdm(os.listdir(TRAIN_PATH)):
    for scan in os.listdir(os.path.join(TRAIN_PATH, patient)):
        scans = []
        for dicom_img in os.listdir(os.path.join(TRAIN_PATH, patient, scan)):
            scans.append(os.path.join(TRAIN_PATH, patient, scan, dicom_img))
        img_paths.append(scans)

        transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256, 256), antialias=True)])

        for folder in img_paths:
          images = []
          patient_id = folder[0].split('/')[-3]

          for elemnts in folder[:limit]:
            img = cv2.imread(elemnts)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255
            images.append(img)

          images = np.stack(images)
          images = torch.tensor(images, dtype = torch.float).unsqueeze(dim = 1)
          image = transform(images).squeeze(dim = 1)
          label = train_data[train_data.patient_id == int(patient_id)].values[0][1:]
          label_list.append(label)
          img_list.append(image)

NameError: ignored

In [ ]:
class Config:
    SEED = 42
    IMAGE_SIZE = [256, 256]
    BATCH_SIZE = 16
    EPOCHS = 10
    TARGET_COLS  = [
       'bowel_healthy', 'bowel_injury', 'extravasation_healthy',
       'extravasation_injury', 'kidney_healthy', 'kidney_low', 'kidney_high',
       'liver_healthy', 'liver_low', 'liver_high', 'spleen_healthy',
       'spleen_low', 'spleen_high', 'any_injury'
    ]

torch.manual_seed(Config.SEED)
config = Config()
len(Config.TARGET_COLS)

14

In [ ]:
# Function to handle the split for each group
def split_group(group, test_size=0.2):
    if len(group) == 1:
        return (group, pd.DataFrame()) if np.random.rand() < test_size else (pd.DataFrame(), group)
    else:
        return train_test_split(group, test_size=test_size, random_state=42)

# Initialize the train and validation datasets
train_data = pd.DataFrame()
val_data = pd.DataFrame()

# Iterate through the groups and split them, handling single-sample groups
for _, group in dataframe.groupby(config.TARGET_COLS):
    train_group, val_group = split_group(group)
    train_data = pd.concat([train_data, train_group], ignore_index=True)
    val_data = pd.concat([val_data, val_group], ignore_index=True)

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=True):
        self.paths = dataframe.image_path.tolist()
        self.labels = dataframe[config.TARGET_COLS].values
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        image = Image.open(self.paths[index]).convert('RGB')
        label = torch.tensor(self.labels[index], dtype=torch.float32)

        if self.transform:
            image = torchvision.transforms.Compose([
                    torchvision.transforms.Resize((256, 256)),
                    torchvision.transforms.RandomResizedCrop(256),   # Random crop and resize
                    torchvision.transforms.RandomHorizontalFlip(),    # Random horizontal flip
                    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
                    )(image)
        else:
          image = torchvision.transforms.Compose([
                    torchvision.transforms.Resize((256, 256)),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
                    )(image)



      # def __getitem__(self, idx):

      #   image_path = self.df['image_path'][idx]
      #   dicom_file = pydicom.dcmread(image_path)
      #   pixel_array = dicom_file.pixel_array.astype(np.int16)
      #   image = Image.fromarray(pixel_array)
      #   image = self.transform(image)
      #   image = image.cuda()

      #   target = self.df[config.TARGET_COLUMNS].iloc[idx]
      #   target = torch.tensor(target.values, dtype=torch.float32)
      #   target = target.cuda()

      #   return image, target[:2], target[2:4], target[4:7], target[7:10], target[10:13]

        return image, label

In [ ]:
# Create the datasets
batch_size = 32

dataset_train = MyDataset(train_data, transform=True)
train_dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = MyDataset(val_data, transform=False)
val_dataloader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

In [ ]:
for img, label in train_dataloader:
  print(img.shape, label.shape)
  break

torch.Size([32, 3, 256, 256]) torch.Size([32, 14])


In [ ]:
#a function to display images
def show_images(images):
    fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
    for idx, image in enumerate(images):
        image = torchvision.transforms.Compose([ torchvision.transforms.Normalize(mean = [ 0., 0., 0. ],std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                                 torchvision.transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ], std = [ 1., 1., 1. ])])(image)

        image = image.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for displaying
        axes[idx].imshow(image)
        axes[idx].axis("off")
    plt.show()

In [ ]:
# Display the sample images
indxs = torch.randint(len(dataset_train), size=(5,))
show_images([dataset_train[i][0] for i in indxs])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# base = models.resnet152(weights=True)
# conv1_weights = base.state_dict()['conv1.weight'].sum(dim=1, keepdim=True)
# base.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# base.state_dict()['conv1.weight'] = conv1_weights

# # output for bowel and extravastaion
# out_be = nn.Sequential(
#     nn.Linear(1000, 512),
#     nn.ReLU(),
#     nn.Dropout(0.5),
#     nn.Linear(512, 2),
#     nn.Softmax(dim=1)
# )

# # output for kidney, liver and spleen
# out_kls = nn.Sequential(
#     nn.Linear(1000, 512),
#     nn.ReLU(),
#     nn.Dropout(0.5),
#     nn.Linear(512, 3),
#     nn.Softmax(dim=1)
# )

# class Model(nn.Module):
#     def __init__(self, base, out_bowel, out_extravasation, out_kidney, out_liver, out_spleen):
#         super(Model, self).__init__()
#         self.base = base
#         self.out_bowel = out_bowel
#         self.out_extravasation = out_extravasation
#         self.out_kidney = out_kidney
#         self.out_liver = out_liver
#         self.out_spleen = out_spleen

#     def forward(self, x):
#         x = self.base(x)
#         out_bowel = self.out_bowel(x)
#         out_extravasation = self.out_extravasation(x)
#         out_kidney = self.out_kidney(x)
#         out_liver = self.out_liver(x)
#         out_spleen = self.out_spleen(x)

#         return out_bowel, out_extravasation, out_kidney, out_liver, out_spleen

# model = Model(base, out_be, out_be, out_kls, out_kls, out_kls)

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self, input_shape=3):
        super(MyModel, self).__init__()
        self.input_shape = input_shape
        self.input = torch.nn.Conv2d(self.input_shape, 3, kernel_size=3)
        self.model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
        # Freeze model parameters
        for param in self.model.parameters():
            param.requires_grad = False

        self.model.fc = torch.nn.Sequential(
            torch.nn.Linear(self.model.fc.in_features, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 32))

        self.bowel = torch.nn.Linear(32, 2)
        self.extra = torch.nn.Linear(32, 2)
        self.liver = torch.nn.Linear(32, 3)
        self.kidney = torch.nn.Linear(32, 3)
        self.spleen = torch.nn.Linear(32, 3)
        self.any_injury = torch.nn.Linear(32, 1)


        for param in self.model.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        if self.input_shape != 3:
          x = self.input(x)
        model_out = self.model(x)
        out1 = torch.nn.functional.softmax(self.bowel(model_out), dim=1)
        out2 = torch.nn.functional.softmax(self.extra(model_out), dim=1)
        out3 = torch.nn.functional.softmax(self.liver(model_out), dim=1)
        out4 = torch.nn.functional.softmax(self.kidney(model_out), dim=1)
        out5 = torch.nn.functional.softmax(self.spleen(model_out), dim=1)
        out6 = torch.nn.functional.softmax(self.any_injury(model_out), dim=1)
        out = torch.cat((out1, out2, out3, out4, out5, out6), dim=1)
        return out

# Instantiate the model
model = MyModel().to(device)
torchsummary.summary(model, (3, 256, 256))

In [ ]:
# Instantiate the model
model = MyModel().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
loss_one = torch.nn.BCEWithLogitsLoss(pos_weight = torch.tensor([6.0]).to(device))
loss_two_bowl = torch.nn.CrossEntropyLoss(weight = torch.tensor([1.0, 2.0])).to(device)
loss_two_extra = torch.nn.CrossEntropyLoss(weight = torch.tensor([1.0, 6.0])).to(device)
loss_three  = torch.nn.CrossEntropyLoss(label_smoothing = 0.05, weight = torch.tensor([1.0, 2.0, 4.0])).to(device)



total_epochs = 5
train_losses = []
val_losses = []
val_accuracies = []
best_loss = 10000

for epoch in range(total_epochs):
    model.train()  # Set the model to training mode
    for train_ind, (images, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()

        # Move data to GPU
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        bowel_loss = loss_two_bowl(outputs[:, 0:2], labels[:, 0:2])
        extravasation_loss = loss_two_extra(outputs[:, 2:4], labels[:, 2:4])
        kidney_loss = loss_three(outputs[:, 4:7], labels[:, 4:7])
        liver_loss = loss_three(outputs[:, 7:10], labels[:, 7:10])
        spleen_loss = loss_three(outputs[:, 10:13], labels[:, 10:13])
        any_injury_loss = loss_one(outputs[:, 13], labels[:, 13])


        loss = bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss + any_injury_loss
        loss.backward()
        optimizer.step()

    # Update learning rate using the scheduler
    # scheduler.step()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_ind, (images, labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            bowel_loss = loss_two_bowl(outputs[:, 0:2], labels[:, 0:2]).item()
            extravasation_loss = loss_two_extra(outputs[:, 2:4], labels[:, 2:4]).item()
            kidney_loss = loss_three(outputs[:, 4:7], labels[:, 4:7]).item()
            liver_loss = loss_three(outputs[:, 7:10], labels[:, 7:10]).item()
            spleen_loss = loss_three(outputs[:, 10:13], labels[:, 10:13]).item()
            any_injury_loss = loss_one(outputs[:, 13], labels[:, 13]).item()


            val_loss += bowel_loss + extravasation_loss + kidney_loss + liver_loss + spleen_loss + any_injury_loss

            predicted = torch.zeros(outputs.size()).to(device)
            for ind, arg in enumerate(torch.argmax(outputs, dim=1)):
              predicted[ind, arg] = 1

            total += labels.size(0) * labels.size(1)  # Total number of predictions
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_dataloader)
    val_accuracy = 100.0 * correct / total
    if val_loss <= best_loss:
      best_loss = val_loss
      torch.save(model.state_dict(), './drive/MyDrive/model_weights.pth')
     # Append loss and accuracy values to lists
    train_losses.append(loss.item())
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    print(f"Epoch [{epoch+1}/{total_epochs}] - Loss: {loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.2f}%")

In [ ]:
torch.save(model.state_dict(), './drive/MyDrive/model_Alexnet_weights.pth')

In [ ]:
# Plot training and validation progress
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Validation Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
model.load_state_dict(torch.load('./drive/MyDrive/DATA/model_weights.pth'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set the model to evaluation mode
model.eval()

# Select a random image from the validation dataset
random_index = np.random.randint(len(dataset_val))
image, label = dataset_val[random_index]

# Move the image to the GPU if available
image = image.to(device)

# Pass the image through the model
with torch.no_grad():
    output = model(image.unsqueeze(0))  # Unsqueeze to add batch dimension

# Convert the output logits to probabilities using sigmoid function
predicted_probs = torch.sigmoid(output)[0]

# Convert predicted probabilities to binary predictions
predicted_labels = (predicted_probs > 0.5).int()

# Display the image, actual labels, and predicted labels

image = torchvision.transforms.Compose([ torchvision.transforms.Normalize(mean = [ 0., 0., 0. ],std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                torchvision.transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],std = [ 1., 1., 1. ])])(image.cpu())
plt.imshow(image.permute(1, 2, 0))  # Move image to CPU and change channel order
#plt.title(f"Actual Labels: {label}\nPredicted Labels: {predicted_labels}")
plt.title(f"Actual Labels: {label}\nPredicted Labels: {predicted_labels}")
plt.show()


In [ ]:
model = torch.load('model.pth')